**Author:** Christina Konstantopoulou  
**Version:** 1.2 

---------------------------------------------------------------
This script:

- automatically downloads the latest list of certified growers 
(as a CSV) from the RSPO PRISMA website,  
- filters the ones with license start dates in a specific time range,
- then downloads their Audit Reports one by one.
- Finally it saves a log of what was downloaded.
  


# Install packages

In [ ]:
# --- Prerequisites ---

# Uncomment the rows below and run this once to install everything:

# !pip install playwright pandas nest_asyncio
# !playwright install

# Import packages

In [1]:
import os
import asyncio
import pandas as pd
from datetime import datetime
import shutil
from playwright.async_api import async_playwright
import nest_asyncio

In [84]:
VERSION = "1.2"
RUN_TIMESTAMP = datetime.now().strftime("%Y-%m-%d %H:%M")

print(f"Script Version: {VERSION} | Run at: {RUN_TIMESTAMP}")


Script Version: 1.2 | Run at: 2025-06-05 16:29


# USER CONFIGURATION - Specify a range for dates and file paths

In [66]:
nest_asyncio.apply()

# === # Define the range for the license start date ===
START_DATE = datetime(2025, 4, 1) # Specifz the licence start date
END_DATE = datetime(2025, 4, 10) 

#CSV_PATH = "C:/Users/Christina/Desktop/RSPO_PNC_data_20250602_120628.csv"
DOWNLOADS_DIR = "C:/Users/Christina/Desktop/PRISMA_downloads"
os.makedirs(DOWNLOADS_DIR, exist_ok=True)

# Define the download directory for the webs
DOWNLOADS_DIR_CSV = "C:/Users/Christina/Desktop/"
os.makedirs(DOWNLOADS_DIR_CSV, exist_ok=True)

# Automatically download all the members list as CSV from the PRISMA website

In [4]:
async def download_csv():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # set headless=True for background mode, headless=False will popup the browser
        context = await browser.new_context(accept_downloads=True)
        page = await context.new_page()

        await page.goto("https://platform.prismabyrspo.org/certificate-registry/certified-growers", timeout=60000)
        await page.wait_for_selector("div:text('Download as CSV')", timeout=10000)

        async with page.expect_download() as download_info:
            await page.click("div:text('Download as CSV')")
        download = await download_info.value

        csv_filename = download.suggested_filename
        csv_path = os.path.join(DOWNLOADS_DIR_CSV, csv_filename)
        await download.save_as(csv_path)
        print(f" CSV downloaded to: {csv_path}")

        await browser.close()
        return csv_path

csv_file_path = await download_csv()

 CSV downloaded to: C:/Users/Christina/Desktop/RSPO_PNC_data_20250603_164859.csv


# Prepare for download

In [75]:
# Delete any existing files in the download folder
for filename in os.listdir(DOWNLOADS_DIR):
    file_path = os.path.join(DOWNLOADS_DIR, filename)
    try:
        if os.path.isfile(file_path):
            os.remove(file_path)
    except Exception as e:
        print(f"Could not delete {file_path}: {e}")

# === Load and  Filter RSPO members from the CSV file within date range ===
df = pd.read_csv(csv_file_path, header=1)
df['License Start Date'] = pd.to_datetime(df['License Start Date'], format="%d-%b-%Y", errors='coerce')
filtered_df = df[
    (df['License Start Date'] >= START_DATE) &
    (df['License Start Date'] <= END_DATE)
]
target_records = filtered_df[['Prisma Trading Account ID', 'License Start Date']].dropna().drop_duplicates()

target_records

,Prisma Trading Account ID,License Start Date
341,TA25-016961,2025-04-02
389,TA25-016982,2025-04-08
399,TA25-016789,2025-04-09
400,TA25-016647,2025-04-08
403,TA25-016835,2025-04-05
419,TA25-017090,2025-04-06
420,TA25-037307,2025-04-10
424,TA25-016993,2025-04-01
432,TA25-017057,2025-04-08
433,TA25-017082,2025-04-09


# Download audit reports

In [76]:
# Track downloaded files
download_log = []

# === Main download function ===
async def download_audit_reports():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless= True)  # set headless=True for background mode, headless=False will popup the browser 
        context = await browser.new_context(accept_downloads=True)
        page = await context.new_page()

        for _, row in target_records.iterrows():
            prisma_id = row['Prisma Trading Account ID'].strip()
            license_start = row['License Start Date']

            print(f"\n Searching for {prisma_id}...")
            await page.goto("https://platform.prismabyrspo.org/certificate-registry/certified-growers", timeout=60000)
            await page.wait_for_selector(".MuiDataGrid-root", timeout=30000)

            try:
                # Find and use search bar
                await page.wait_for_selector("input#search", timeout=10000)
                search_box = await page.query_selector("input#search")
                if not search_box:
                    print(f" Search bar not found for {prisma_id}")
                    continue

                # Clear and enter PO ID
                await search_box.click()
                await search_box.fill("")
                await search_box.type(prisma_id)
                await page.keyboard.press("Enter")
                await page.evaluate("document.activeElement.blur()")  # trigger search
                await page.wait_for_timeout(2000)

                # Verify that top row matches the searched PO ID
                rows = await page.query_selector_all(".MuiDataGrid-row")
                if not rows:
                    print(f"  No search results for {prisma_id}")
                    continue

                first_row_cells = await rows[0].query_selector_all(".MuiDataGrid-cell")
                if len(first_row_cells) < 6:
                    print(f"  Unexpected row format for {prisma_id}")
                    continue

                visible_po_id = (await first_row_cells[5].inner_text()).strip()
                print(f"  PO ID: {visible_po_id}")
                if visible_po_id != prisma_id:
                    print(f"  Mismatch — skipping {prisma_id}")
                    continue

                # Click and download audit reports
                await rows[0].click()
                await page.wait_for_timeout(1500)

                elements = await page.query_selector_all("text=Audit Report")
                if not elements:
                    print(f"  No Audit Reports for {prisma_id}")
                    continue

                for j, el in enumerate(elements):
                    try:
                        async with page.expect_download(timeout=5000) as download_info:
                            await el.click()
                        download = await download_info.value
                        filename = f"{prisma_id}_{license_start.strftime('%Y-%m-%d')}_audit_{j+1}.pdf"
                        save_path = os.path.join(DOWNLOADS_DIR, filename)
                        await download.save_as(save_path)
                        print(f"  Saved: {save_path}")
                        download_log.append({
                            'Prisma ID': prisma_id,
                            'License Start Date': license_start.strftime('%Y-%m-%d'),
                            'File Type': 'Audit Report',
                            'Filename': filename
                        })
                    except Exception as e:
                        print(f"  Download failed for {prisma_id}: {e}")

            except Exception as e:
                print(f"  Error processing {prisma_id}: {e}")
                continue

        await browser.close()

        # Save download log
        df_log = pd.DataFrame(download_log)
        log_path = os.path.join(DOWNLOADS_DIR, "download_log.csv")
        df_log.to_csv(log_path, index=False)
        print(f"\n Download log saved to: {log_path}")

# === Run script ===
asyncio.run(download_audit_reports())


 Searching for TA25-016961...
  PO ID: TA25-016961
  No Audit Reports for TA25-016961

 Searching for TA25-016982...
  PO ID: TA25-016982
  Saved: C:/Users/Christina/Desktop/PRISMA_downloads\TA25-016982_2025-04-08_audit_1.pdf

 Searching for TA25-016789...
  PO ID: TA25-016789
  Saved: C:/Users/Christina/Desktop/PRISMA_downloads\TA25-016789_2025-04-09_audit_1.pdf

 Searching for TA25-016647...
  PO ID: TA25-016647
  Saved: C:/Users/Christina/Desktop/PRISMA_downloads\TA25-016647_2025-04-08_audit_1.pdf

 Searching for TA25-016835...
  PO ID: TA25-016835
  Saved: C:/Users/Christina/Desktop/PRISMA_downloads\TA25-016835_2025-04-05_audit_1.pdf

 Searching for TA25-017090...
  PO ID: TA25-017090
  Saved: C:/Users/Christina/Desktop/PRISMA_downloads\TA25-017090_2025-04-06_audit_1.pdf

 Searching for TA25-037307...
  PO ID: TA25-037307
  Saved: C:/Users/Christina/Desktop/PRISMA_downloads\TA25-037307_2025-04-10_audit_1.pdf

 Searching for TA25-016993...
  PO ID: TA25-016993
  Saved: C:/Users/Chr

In [24]:
members = {}

async def extract_member_list():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto("https://platform.prismabyrspo.org/certificate-registry/certified-growers", timeout=60000)
        await page.wait_for_selector(".MuiDataGrid-root", timeout=30000)

        # Let the table load
        await page.wait_for_timeout(2000)

        rows = await page.query_selector_all(".MuiDataGrid-row")
        print(f" Found {len(rows)} rows.")

        for row in rows:
            try:
                cells = await row.query_selector_all(".MuiDataGrid-cell")
                if len(cells) >= 1:
                    name_cell = cells[0]
                    member_name = (await name_cell.inner_text()).strip()

                    # Try to extract a profile link
                    link_element = await name_cell.query_selector("a")
                    member_link = await link_element.get_attribute("href") if link_element else "No link"

                    members[member_name.lower()] = member_link
            except Exception as e:
                print(f" Error parsing row: {e}")

        await browser.close()

await extract_member_list()


 Found 0 rows.


In [9]:
# Search by name (partial match, case-insensitive)
search_name = input("Enter part of the member's name: ").lower()

found_members = [name for name in members if search_name in name]

if found_members:
    for name in found_members:
        print(f" Member Name: {name}")
        print(f" Link: {members[name]}")
else:
    print(f" No matches found for: {search_name}")



Enter part of the member's name:  Extractora del Sur de Casanare S.A.S.


 Member Name: extractora del sur de casanare s.a.s.
 Link: https://rspo.org/members/1-0214-16-000-00
